In [2]:
import jax.numpy as jnp
# from ckks.encryptor import Encryptor
from ckks.encoder import Encoder
from ckks.utils import get_modulo

In [3]:
M = 32
scale = 2**10

In [4]:
message = jnp.array([777,666,555,444,333,222,111,0])
encoder = Encoder(M, scale, 0)
# encryptor = Encryptor(13, M, 256, seed=0)

In [5]:
code = encoder.encode(message)

In [6]:
jnp.around(encoder.decode(code)).astype(int)

/home/jiajie/OSS/jax/jax/_src/lax/lax.py:487: ComplexWarning: Casting complex values to real discards the imaginary part
  return _convert_element_type(operand, new_dtype, weak_type=False)


DeviceArray([777, 666, 555, 444, 333, 222, 111,   0], dtype=int32)

In [8]:
m1 = jnp.arange(111,999,111)
m2 = jnp.arange(999,111,-111)
m1, m2

(DeviceArray([111, 222, 333, 444, 555, 666, 777, 888], dtype=int32),
 DeviceArray([999, 888, 777, 666, 555, 444, 333, 222], dtype=int32))

In [9]:
m1 = jnp.arange(1, 9, 1)
m2 = jnp.arange(9, 1,-1)
m1, m2

(DeviceArray([1, 2, 3, 4, 5, 6, 7, 8], dtype=int32),
 DeviceArray([9, 8, 7, 6, 5, 4, 3, 2], dtype=int32))

In [10]:
e1, e2 = encoder.encode(m1), encoder.encode(m2)